Praktikum 9 -ImageReconstruction

In [5]:
import os
import numpy as np
from scipy.interpolate import interp1d
from scipy.ndimage import rotate
import glob
import cv2
from imutils import paths
import imutils

## Aufgabe 1

In [6]:
RAW_ROWS = 227
RAW_COLUMNS = 198
RAW_LEVELS = 129

In [7]:
DIRECTORY_PATH: str = os.getcwd()

In [4]:
def load_raw_file(file_name: str, levels, rows, columns):
    raw = np.fromfile(DIRECTORY_PATH + file_name, dtype='int8')
    numpy_raw = np.reshape(raw, (levels, rows, columns))
    return numpy_raw

In [5]:
# Alternative Art MIP auszurechnen, ohne numpy -> Ist jedoch deutlich langsamer, weswegen wir die numpy Alternative verwendet haben.
def custom_mip_calc(raw, row_count, column_count):
    flat_image = np.zeros((row_count, column_count), dtype="int8")
    for z in range(raw.shape[0]):
        for y in range(raw.shape[1]):
            for x in range(raw.shape[2]):
                if raw[z, y, x] > flat_image[y, x]:
                    flat_image[y, x] = raw[z, y, x]
    return flat_image

In [6]:
def save_xy_middle_slice(raw):
    save_to_pgm(raw[int(RAW_LEVELS / 2), :, :], RAW_COLUMNS, RAW_ROWS, "xy_middle_slice")

In [7]:
def save_xz_middle_slice(raw):
    save_to_pgm(raw[:, int(RAW_ROWS / 2), :], RAW_COLUMNS, RAW_LEVELS, "xz_middle_slice")

In [8]:
def save_yz_middle_slice(raw):
    save_to_pgm(raw[:, :, int(RAW_COLUMNS / 2)], RAW_ROWS, RAW_LEVELS, "yz_middle_slice")

In [9]:
def slices(raw):
    save_xy_middle_slice(raw)
    save_xz_middle_slice(raw)
    save_yz_middle_slice(raw)

In [10]:
def save_mip(raw):
    save_to_pgm(np.max(raw, axis=0), RAW_COLUMNS, RAW_ROWS, "maximum_intensity")

In [11]:
def save_to_pgm(image_array, width, height, file_name, folder="created_images_Aufgabe_1"):
    pgm_header = "P5\n" + " " + str(width) + " " + str(height) + " " + str(255) + "\n"
    file = open(DIRECTORY_PATH + "/" + folder + "/" + file_name + ".pgm", 'wb')
    pgm_header_byte = bytearray(pgm_header, 'utf-8')
    file.write(pgm_header_byte)
    file.write(bytearray(image_array))
    file.close()

In [12]:
raw = load_raw_file("/images_for_reconstruction/whatisit_129x227x198_8bit.raw",
                    RAW_LEVELS,
                    RAW_ROWS,
                    RAW_COLUMNS)

slices(raw)
save_mip(raw)

## Aufgabe 2

In [13]:
ARTEMISIA_RAW_ROWS = 138
ARTEMISIA_RAW_COLUMNS = 138
ARTEMISIA_RAW_LEVELS = 145

In [14]:
raw = load_raw_file("/images_for_reconstruction/Artemisia_pollen_145x138x138_8bit.raw",
                    ARTEMISIA_RAW_LEVELS,
                    ARTEMISIA_RAW_ROWS,
                    ARTEMISIA_RAW_COLUMNS)

In [15]:
# Das sind tests um zu validieren, ob das Rotieren funktioniert.
rotated = rotate(raw, angle=200, mode='nearest', axes=(1, 2), reshape=False, order=0)
print("OUTPUT: " + str(np.max(rotated)))
print("OUTPUT_SHAPE: " + str(rotated.shape))
save_to_pgm(rotated[int(ARTEMISIA_RAW_LEVELS / 2), :, :], ARTEMISIA_RAW_COLUMNS, ARTEMISIA_RAW_ROWS, "xy_pollen_test_3",
            "created_movie_frames_Aufgabe_2/")

OUTPUT: 127
OUTPUT_SHAPE: (145, 138, 138)


In [16]:
def generate_movie_frames(raw, degree_step: int = 1):
    if 360 % degree_step != 0:
        raise ValueError('Degree step is invalid.')

    rotated_images = []
    for degree in range(int(360 / degree_step)):
        degree_string = str(degree * degree_step)
        print("DEGREE: " + degree_string)

        movie_frame_name = "movie_frame_"
        if len(degree_string) == 1:
            movie_frame_name += movie_frame_name + "00" + degree_string
        elif len(degree_string) == 2:
            movie_frame_name += movie_frame_name + "0" + degree_string
        else:
            movie_frame_name += movie_frame_name + degree_string

        rotated_image = rotate(raw, angle=degree * degree_step, mode='nearest', axes=(1, 2), reshape=False, order=0)
        rotated_image_mip = np.max(rotated_image, axis=1).astype('int8')
        rotated_images.append(rotated_image_mip)
        save_to_pgm(rotated_image_mip,
                    ARTEMISIA_RAW_COLUMNS,
                    ARTEMISIA_RAW_ROWS,
                    movie_frame_name,
                    "created_movie_frames_Aufgabe_2")

    return rotated_images

In [17]:
def create_video(file_name: str = "pollen", folder_name: str = "/created_movie_frames_Aufgabe_2"):
    movie_frames = []
    [movie_frames.append(movie_frame) for movie_frame in
     sorted(glob.glob(DIRECTORY_PATH + folder_name + "/movie_frame*.pgm"))]

    #out = cv2.VideoWriter(file_name + '.avi', cv2.VideoWriter_fourcc(*'DIVX'), 15,
    #                     (ARTEMISIA_RAW_ROWS, ARTEMISIA_RAW_COLUMNS))
    out = cv2.VideoWriter('video.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 15,
                          (ARTEMISIA_RAW_ROWS, ARTEMISIA_RAW_COLUMNS))
    for movie_frame in movie_frames:
        out.write(cv2.imread(movie_frame))
    out.release()

In [18]:
generate_movie_frames(raw)
create_video()

DEGREE: 0
DEGREE: 1
DEGREE: 2
DEGREE: 3
DEGREE: 4
DEGREE: 5
DEGREE: 6
DEGREE: 7
DEGREE: 8
DEGREE: 9
DEGREE: 10
DEGREE: 11
DEGREE: 12
DEGREE: 13
DEGREE: 14
DEGREE: 15
DEGREE: 16
DEGREE: 17
DEGREE: 18
DEGREE: 19
DEGREE: 20
DEGREE: 21
DEGREE: 22
DEGREE: 23
DEGREE: 24
DEGREE: 25
DEGREE: 26
DEGREE: 27
DEGREE: 28
DEGREE: 29
DEGREE: 30
DEGREE: 31
DEGREE: 32
DEGREE: 33
DEGREE: 34
DEGREE: 35
DEGREE: 36
DEGREE: 37
DEGREE: 38
DEGREE: 39
DEGREE: 40
DEGREE: 41
DEGREE: 42
DEGREE: 43
DEGREE: 44
DEGREE: 45
DEGREE: 46
DEGREE: 47
DEGREE: 48
DEGREE: 49
DEGREE: 50
DEGREE: 51
DEGREE: 52
DEGREE: 53
DEGREE: 54
DEGREE: 55
DEGREE: 56
DEGREE: 57
DEGREE: 58
DEGREE: 59
DEGREE: 60
DEGREE: 61
DEGREE: 62
DEGREE: 63
DEGREE: 64
DEGREE: 65
DEGREE: 66
DEGREE: 67
DEGREE: 68
DEGREE: 69
DEGREE: 70
DEGREE: 71
DEGREE: 72
DEGREE: 73
DEGREE: 74
DEGREE: 75
DEGREE: 76
DEGREE: 77
DEGREE: 78
DEGREE: 79
DEGREE: 80
DEGREE: 81
DEGREE: 82
DEGREE: 83
DEGREE: 84
DEGREE: 85
DEGREE: 86
DEGREE: 87
DEGREE: 88
DEGREE: 89
DEGREE: 90
DEGREE: 9

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [19]:
def custom_rotation():
    # todo ->  muss noch implementiert werden (Es funktioniert jetzt zwar aber ist halt mit ner Library)
    a = 2


In [ ]:
# def transform_array(src_array):
#     trgPos = lev, row, col, 1
#     srcPos = myproduct(invMat, trgPos)
#     trgArr(lev, row, col) = interpolNN(src_array, srcPos)

In [47]:
# def rotate_test(raw, degree):
#     # todo delete
#     rotation_y_axis = np.asarray([
#         [np.cos(degree), 0, np.sin(degree), 0],
#         [0, 1, 0, 0],
#         [-np.sin(degree), 0, np.cos(degree), 0],
#         [0, 0, 0, 1]
#     ])
#
#     rotated = rotate(raw, angle=5)
#     return rotated
#
#     inverse = np.linalg.inv(raw)
#     trg_arr = np.zeros(raw.shape)
#
#     #Iteration LEVELS
#     for lev_index, lev in enumerate(raw):
#         #Iteration ROWS
#         for row_index, row in enumerate(lev):
#             #Iteration COLUMNS
#             for col_index, col in enumerate(row):
#                 # l = raw[lev_index, row_index, col_index]
#                 src_pos = lev_index, row_index, col_index, 1
#                 trg_pos = np.matmul(inverse, src_pos)
#                 trg_arr[lev_index, row_index, col_index] = interp1d(raw, trg_pos)
#     return trg_arr

In [ ]:
# def rotate_c(raw, deg_angle):
#     angle = np.radians(deg_angle)
#
#     rotation_matrix = np.asarray([
#         [np.cos(angle), 0, np.sin(angle)],
#         [0, 1, 0],
#         [-np.sin(angle), 0, np.cos(angle)]
#     ])
#
#     d = raw.shape[0]
#     h = raw.shape[1]
#     w = raw.shape[2]
#
#     min_new_x = 0
#     max_new_x = 0
#     min_new_y = 0
#     max_new_y = 0
#     min_new_z = 0
#     max_new_z = 0
#     new_coords = []
#
#     output = np.zeros((d, w, h), dtype="int8")
#     cos = np.cos(angle)
#     sin = np.sin(angle)
#
#     for z in range(d):
#         for y in range(h):
#             for x in range(w):
#                 new_x = int(z * cos + x * sin)
#                 new_y = y
#                 new_z = int(z * -sin + x * cos)
#
#                 if new_x >= ARTEMISIA_RAW_COLUMNS:
#                     print("BIGGER: " + str(new_x))
#                     continue
#
#                 output[new_x, new_y, new_x] = raw[z, y, x]
#
#                 val = raw[z, y, x]
#                 new_coords.append((val, x, y, z))
#                 if new_x < min_new_x: min_new_x = new_x
#                 if new_x > max_new_x: max_new_x = new_x
#                 if new_y < min_new_y: min_new_y = new_y
#                 if new_y > max_new_y: max_new_y = new_y
#                 if new_z < min_new_z: min_new_z = new_z
#                 if new_z > max_new_z: max_new_z = new_z
#
#     new_x_offset = abs(min_new_x)
#     new_y_offset = abs(min_new_y)
#     new_z_offset = abs(min_new_z)
#     #
#     new_width = abs(min_new_x - max_new_x)
#     new_height = abs(min_new_y - max_new_y)
#     new_depth = abs(min_new_z - max_new_z)
#
#     rotated = np.zeros((d, h, w), dtype='int8')
#     for coord in new_coords:
#         val = coord[0]
#         x = coord[1]
#         y = coord[2]
#         z = coord[3]
#         if rotated[z][y][x] == 0:
#             rotated[z][y][x] = val
#
#     save_to_PGM(output.astype("int8")[:, :, int(ARTEMISIA_RAW_COLUMNS / 2)], ARTEMISIA_RAW_ROWS, ARTEMISIA_RAW_LEVELS, "output_rotate_with_custom")

In [ ]:
# def rotate_custom(raw, degree):
#     rotation_matrix = np.asarray([
#         [np.cos(degree), 0, np.sin(degree), 0],
#         [0, 1, 0, 0],
#         [-np.sin(degree), 0, np.cos(degree), 0],
#         [0, 0, 0, 1]
#     ])
#
#     # compute the mid point
#     midpoint = np.mean(raw, 0)
#
#     # let's ignore the y coordinate
#     midpoint[1] = 0.0
#
#     # translate all points so that midpoint is at [0, y, 0]
#     translated = raw - midpoint
#
#     # for use in the following step
#     cos = np.cos(degree)
#     sin = np.sin(degree)
#
#     # apply the rotation matrix
#     rotated = [np.array([v[0] * cos + v[2] * sin, v[1], -v[0] * sin + v[2] * cos]) for v in translated]
#
#     save_to_PGM(rotated[:, :, int(ARTEMISIA_RAW_COLUMNS / 2)], ARTEMISIA_RAW_ROWS, ARTEMISIA_RAW_LEVELS, "output_rotate_with_custom")
#
#     # translate back
#     untranslated = rotated + midpoint
#
#     save_to_PGM(untranslated[:, :, int(ARTEMISIA_RAW_COLUMNS / 2)], ARTEMISIA_RAW_ROWS, ARTEMISIA_RAW_LEVELS, "output_rotate_with_custom")
#     return untranslated

## Aufgabe 3

In [8]:
def create_panorama(path_to_images, custom_images: bool):
    # initialize images
    image_paths = sorted(list(paths.list_images(path_to_images)))
    images = []
    # load images
    for image_path in image_paths:
        images.append(cv2.imread(image_path))

    #initializing image stitcher
    print("stitching images...")
    stitcher = cv2.createStitcher() if imutils.is_cv3() else cv2.Stitcher_create()
    (status, stitched) = stitcher.stitch(images)

    #check if stitching worked
    if status == 0:
        print("cropping...")
        stitched = cv2.copyMakeBorder(stitched, 10, 10, 10, 10,
                                      cv2.BORDER_CONSTANT, (0, 0, 0))
        # convert to gray-scale and threshold
        gray = cv2.cvtColor(stitched, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]

        # find the largest contour to get outline of stitched image
        contours = cv2.findContours(thresh.copy(),
                                    cv2.RETR_EXTERNAL,
                                    cv2.CHAIN_APPROX_SIMPLE)

        contours = imutils.grab_contours(contours)
        (x, y, w, h) = cv2.boundingRect(array=max(contours, key=cv2.contourArea))

        mask = np.zeros(thresh.shape, dtype="uint8")
        cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)
        min_rect = mask.copy()
        sub = mask.copy()

        while cv2.countNonZero(sub) > 0:
            # detect minimal mask for output image
            min_rect = cv2.erode(min_rect, None)
            sub = cv2.subtract(min_rect, thresh)

        contours = cv2.findContours(min_rect.copy(),
                                    cv2.RETR_EXTERNAL,
                                    cv2.CHAIN_APPROX_SIMPLE)

        contours = imutils.grab_contours(contours)
        (x, y, w, h) = cv2.boundingRect(array=max(contours, key=cv2.contourArea))
        # get output from minimal mask and stitched image
        stitched = stitched[y:y + h, x:x + w]
        if custom_images:
            output_file_path = DIRECTORY_PATH + "/result-custom-images" + ".jpg"
        else:
            output_file_path = DIRECTORY_PATH + "/result-given-images" + ".jpg"
        print("Writing result to: " + output_file_path)
        cv2.imwrite(output_file_path, stitched)
        cv2.imshow("Stitched", stitched)
        cv2.waitKey(0)
    else:
        print("Error({})".format(status))

In [9]:
path_to_custom_images_directory = DIRECTORY_PATH + "/panorama_images"
create_panorama(path_to_custom_images_directory, True)
path_to_given_images_directory = DIRECTORY_PATH + "/images_for_blending"
create_panorama(path_to_given_images_directory, False)

stitching images...
cropping...
Writing result to: /Users/john/Downloads/Praktikum_9/result-custom-images.jpg
stitching images...
cropping...
Writing result to: /Users/john/Downloads/Praktikum_9/result-given-images.jpg
